In [77]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


In [93]:

# שלב 1: טעינת הנתונים
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
submission_data = pd.read_csv('sample_submission.csv')

# שמירת עמודת ה-ID
train_ids = train_data['Id']
test_ids = test_data['Id']

In [94]:

# שלב 2: הסרת עמודות עם פחות מ-100 ערכים שאינם NaN
train_data = train_data.dropna(thresh=100, axis=1)
test_data = test_data[train_data.columns.drop('SalePrice', errors='ignore')]  # שמירת אותן עמודות גם ב-test data למעט עמודת המחיר


In [95]:
# שלב 3: חישוב קורלציות עם המחיר ומחיקת עמודות מספריות עם קורלציה נמוכה
numerical_features = train_data.select_dtypes(include=['int64', 'float64']).columns
correlation_matrix = train_data[numerical_features].corr()
price_correlation = correlation_matrix['SalePrice'].abs().sort_values(ascending=False)
threshold = 0.1  # סף לקורלציה נמוכה
high_corr_features = price_correlation[price_correlation > threshold].index

# שמירה על העמודות המספריות עם קורלציה גבוהה והוספת העמודות הקטגוריאליות
categorical_features = train_data.select_dtypes(include=['object']).columns
selected_features = list(high_corr_features) + list(categorical_features)


In [96]:
# הוספת עמודת ה-ID
selected_features = ['Id'] + selected_features + ['SalePrice']
train_data = train_data[selected_features]
selected_features_noprice = [feature for feature in selected_features if feature != 'SalePrice']
test_data = test_data[[feature for feature in selected_features_noprice if feature in test_data.columns]]


In [97]:
# התאמת העמודות ב-test_data לפי train_data
missing_cols = set(selected_features_noprice) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0  # נניח ערך ברירת מחדל

test_data = test_data[selected_features_noprice]


In [98]:
# שלב 4: הסרת ערכים חריגים (Outliers) עבור עמודות מספריות בלבד הקיימות בנתונים
valid_numerical_features = [feature for feature in high_corr_features if feature in train_data.columns]
Q1 = train_data[valid_numerical_features].quantile(0.25)
Q3 = train_data[valid_numerical_features].quantile(0.75)
IQR = Q3 - Q1
outlier_condition = ~((train_data[valid_numerical_features] < (Q1 - 1.5 * IQR)) | (train_data[valid_numerical_features] > (Q3 + 1.5 * IQR))).any(axis=1)
train_data = train_data[outlier_condition]

In [99]:
# הסרת עמודות כפולות אם קיימות
train_data = train_data.loc[:,~train_data.columns.duplicated()]


In [100]:
# שלב 5: הפרדת משתנים תלויים ובלתי תלויים
X = train_data.drop(['SalePrice'], axis=1)
y = train_data['SalePrice'].values.ravel()  # ודא כי y חד-ממדי


In [102]:
len(y)

777

In [103]:
# שלב 6: חלוקה לסט אימון וסט אימות
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


In [104]:
# הסרת עמודת ה-ID לאחר החלוקה לסטים
X_train = X_train.drop(['Id'], axis=1)
X_valid = X_valid.drop(['Id'], axis=1)
test_data_no_id = test_data.drop(['Id'], axis=1)


In [105]:
# עדכון משתנים מספריים וקטגוריאליים אחרי הסינון
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns


In [106]:

# שלב 7: יצירת Pipelines לטיפול בערכים חסרים וקטגוריאליים ולהוספת TruncatedSVD
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # מילוי ערכים חסרים במספר ממוצע
    ('scaler', StandardScaler()),  # סטנדרטיזציה
    ('transformer', PowerTransformer())  # שינוי התפלגות הנתונים ללוגריתם או תפלגות אחרת
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # מילוי ערכים חסרים בערך הנפוץ ביותר
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # קידוד OneHot לערכים קטגוריאליים
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

# הוספת TruncatedSVD לצמצום ממדיות ולאחר מכן MinMaxScaler לנרמול
pipeline_with_svd = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svd', TruncatedSVD(n_components=50)),  # צמצום למספר רכיבים מתאים
    ('scaler', MinMaxScaler())  # נרמול
])



In [107]:
# שלב 8: עיבוד הנתונים עם TruncatedSVD ו-MinMaxScaler
X_train_processed = pipeline_with_svd.fit_transform(X_train)
X_valid_processed = pipeline_with_svd.transform(X_valid)
X_test_processed = pipeline_with_svd.transform(test_data_no_id)


In [109]:

# פונקציה לכוונון היפר-פרמטרים
def tune_model(model, param_grid):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train_processed, y_train)
    return grid_search.best_estimator_


In [110]:

# שלב 9: הגדרת המודלים וכוונון היפר-פרמטרים
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": tune_model(DecisionTreeRegressor(), {'max_depth': [5, 10, 20, None]}),
    "SVM": tune_model(SVR(), {'C': [0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf']}),
    "AdaBoost": tune_model(AdaBoostRegressor(), {'n_estimators': [50, 100, 200]}),
    "K-Nearest Neighbors": tune_model(KNeighborsRegressor(), {'n_neighbors': [3, 5, 10]}),
    "Random Forest": tune_model(RandomForestRegressor(), {'n_estimators': [100, 200, 300], 'max_depth': [5, 10, 20, None]}),
    "Gradient Boosting": tune_model(GradientBoostingRegressor(), {'n_estimators': [100, 200, 300], 'learning_rate': [0.01, 0.1, 0.2]}),
    # "XGBoost": tune_model(xgb.XGBRegressor(), {'n_estimators': [100, 200, 300], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]})
}


In [ ]:

# הרצת המודלים והדפסת התוצאות
results = {}
for name, model in models.items():
    model.fit(X_train_processed, y_train)
    predictions = model.predict(X_valid_processed)
    mse = mean_squared_error(y_valid, predictions)
    r2 = r2_score(y_valid, predictions)
    results[name] = {"MSE": mse, "R2": r2}
    print(f"{name} - MSE: {mse:.2f}, R2: {r2:.2f}")



Linear Regression - MSE: 413705139.69, R2: 0.84
Decision Tree - MSE: 631609300.62, R2: 0.75
SVM - MSE: 879887007.31, R2: 0.66
AdaBoost - MSE: 457580502.84, R2: 0.82
K-Nearest Neighbors - MSE: 798754572.31, R2: 0.69
Random Forest - MSE: 435194126.95, R2: 0.83


In [ ]:
# שימוש ב-ID והכנת הקובץ להגשה
best_model = models["XGBoost"]  # נניח שהמודל הטוב ביותר הוא XGBoost
final_predictions = best_model.predict(X_test_processed)

submission = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': final_predictions
})

submission.to_csv('submission.csv', index=False)

# הצגת הקורלציות
print("קורלציות עם מחיר הדירה:")
print(price_correlation)

# הצגת מפת חום של הקורלציות
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Numerical Features')
plt.show()
